### Generic imports and delcaring the cursor

In [1]:
import sqlite3
import pandas as pd
from IPython.core.magic import register_cell_magic

# Connect to SQLite database (or create it)
conn = sqlite3.connect('messages.db')
cursor = conn.cursor()

In [2]:
# Create table
cursor.execute('''
CREATE TABLE IF NOT EXISTS messages (
    user_id TEXT,
    user_name TEXT,
    date TEXT,
    message_sent INTEGER,
    message_received INTEGER
)
''')

# Insert data
data = [
    ('324A33', 'Sunny Kim', '2020-10-01', 10, 20),
    ('314A33', 'Bobby Tim', '2020-10-01', 50, 10),
    ('784B33', 'Andrew Harp', '2020-10-01', 30, 5),
    ('364E34', 'Andrew Lean', '2020-10-02', 10, 0),
    ('114A33', 'Jalinn Kim', '2020-10-01', 20, 100),
    ('444B33', 'Sam Turner', '2020-10-01', 40, 120)
]

cursor.executemany('''
INSERT INTO messages (user_id, user_name, date, message_sent, message_received)
VALUES (?, ?, ?, ?, ?)
''', data)

# Commit and close
conn.commit()

### Having a function which will allow us to run any sql query in a cell in a jupyter notebook

In [3]:
@register_cell_magic
def sql(line, cell):
    sql_query = cell.strip()
    cursor.execute(sql_query)
    rows = cursor.fetchall()
    df = pd.DataFrame(rows, columns=[desc[0] for desc in cursor.description])
    display(df)

In [4]:
%%sql
SELECT * FROM messages where user_id = '324A33';

,user_id,user_name,date,message_sent,message_received
0,324A33,Sunny Kim,2020-10-01,10,20
1,324A33,Sunny Kim,2020-10-01,10,20
2,324A33,Sunny Kim,2020-10-01,10,20


In [18]:
%%sql
select user_id, max_message_sent_received_ratio from
(
    select
        user_id
        , max(message_sent_received_ratio) as max_message_sent_received_ratio
    from 
    (
        select
            *
            , (message_sent / coalesce(message_received, 0)) as message_sent_received_ratio
        from messages
    )
    group by 1
)
where max_message_sent_received_ratio is not null
order by max_message_sent_received_ratio desc
limit 1;

,user_id,max_message_sent_received_ratio
0,784B33,6
